# Math 134C Week 4

In [3]:
import math

def NPV(r, cf, continuous_compounding = False, dt = None): 
    if dt is None: dt = range(len(cf))
    pv = 0
    if not continuous_compounding:
        for k in range(0, len(cf)):
            pv += cf[k] / (1+r)**dt[k]
    else:
        for k in range(0, len(cf)):
            pv += cf[k] * math.exp(-r * dt[k])
    return pv

# r is the risk-free rate, F is a cash flow, and dT is the time that we get each cash flow
def PV(F, dT, r):
    pv = 0
    if len(dT) != len(F):
        print("Input Error")
        return 0
    for n in range(0, len(F)):
        pv += F[n] * math.exp(-r * dT[n])
    return pv
        
# C_t is the price of call option, S_t is the current stock price, K is the strike, r is the risk-free rate, dt is time to expiration. D is the dividend of the stock, and dT is the time at which we receive the dividend. The delta is for the continuous dividend paying model. The function will return the price of the corresponding put option. 
def put_given_call(C_t, S_t, K, r, dt, D= None, dT = None, delta = 0):
    if D is None: D = []
    if dT is None: dT = []
    return C_t - (S_t - PV(D, dT, r)) * math.exp(-delta * dt) + K * math.exp(-r * dt)

def call_given_put(P_t, S_t, K, r, dt, D = None, dT = None, delta = 0):
    if D is None: D = []
    if dT is None: dT = []
    return P_t + (S_t - PV(D, dT, r)) * math.exp(-delta * dt) - K * math.exp(-r * dt)

def one_step_binomial(r, dt, u, d, Vu, Vd, delta = 0):
    q = (math.exp((r - delta) * dt) - d)/(u-d)
    if not (1 > q > 0): 
        print("Arbitrage opportunity exists")
        return -1
    return math.exp(-r * dt)*(q*Vu+(1-q)*Vd)    

def Delta(r, dt, u, d, Vu, Vd, S, delta = 0):
    return math.exp(-delta * dt)*(Vu-Vd)/(S*(u-d))

def B(r, dt, u, d, Vu, Vd):
    return math.exp(-r * dt)*(u * Vd - d * Vu)/(u-d)

**Quiz 3.1**: 
A newborn baby receives 2,000 on her birthday from her parents which is deposited into an account and invested in the Vanguard S\&P 500 Index Fund. That is 2,000 deposited at t=0.

Assume that on every subsequent birthday up to and including her 16th birthday, the baby's parents deposit an additional 1,000 into the same account and invest the money in the Vanguard S\&P 500 Index Fund.  That is 1,000 deposited on each of t=1 through t=16.  There are no more contributions.

Assume the investments in the Vanguard S\&P 500 Index Fund grow at 6% every year after all taxes and expenses. What will be the balance in the account at the baby's 65th birthday, rounded to whole dollars?

In [5]:
cashflow = [2000] + [1000] * 16
print(f"The balance in the baby's 65th birthday is {NPV(0.06, cashflow) * (1.06) ** 65:.0f}")

The balance in the baby's 65th birthday is 534414


**Quiz 3.2:**
Suppose that the risk-less interest rate is zero.  Suppose that a stock is priced at 100, and one year from now will be at either 130 or 70, with probabilities 0.80 and 0.20, respectively. There are no dividends. 

What is the value of a one-year European call with strike price 110?

In [7]:
u = 130/100
d = 70/100
q = (1-d)/(u-d)
print(f"The price of the call option is {q * 20 + (1-q) * 0:.2f}")

The price of the call option is 10.00


**Quiz 3.3:**
Suppose that the riskless interest rate is zero.  Suppose that a stock is priced at 100, and one year from now will be at either 130 or 70, with probabilities 0.80 and 0.20, respectively. There are no dividends.  Consider a one-year European call option with strike price 110.

What portfolio of the stock and a riskless bond exactly replicates this call option?

In [12]:
print(f"We need to buy {Delta(r = 0, dt = 1, u = 1.3, d = 0.7, Vu = 20, Vd = 0, S = 100):.2f} shares of the stock")
print(f"We also need to short {-B(r = 0, dt = 1, u = 1.3, d =0.7, Vu = 20, Vd = 0):.2f} in the bank")

We need to buy 0.33 shares of the stock
We also need to short 23.33 in the bank


**Quiz 3.4:**
Suppose that the riskless interest rate is zero.  Suppose that a stock is priced at 100, and one year from now will be at either 130 or 70, with probabilities 0.80 and 0.20, respectively. There are no dividends. 

What is $\alpha$, the continuously compounded expected return on the stock?

In [13]:
print(f"The alpha is {math.log(118/100, math.e)*100:.2f}%")

The alpha is 16.55%


**Quiz 3.5:**
Consider a two-period binomial model, where each period is 6 months. Assume the stock price is 46.00, $\sigma$ = 0.28, the annual continuously compounded risk free rate r = 6\% and the annual continuously compounded dividend yield $\delta$ is 2%. 

Assume the binomial tree is constructed using:
$$u = e^{(r-\delta)h+\sigma\sqrt{h}}$$
$$d = e^{(r-\delta)h-\sigma\sqrt{h}}$$


At what strike price would early exercise occur with an American call option?

In [21]:
u = math.exp((0.06-0.02)*0.5 + 0.28 * 0.5 ** 0.5)
d = math.exp((0.06-0.02)*0.5 - 0.28 * 0.5 ** 0.5)
K = 19
Cu = one_step_binomial(r = 0.06, dt = 0.5, u = u, d = d, Vu = max(46 * u * u - K, 0), Vd = max(46 * u * d -K, 0), delta = 0.02)
print(f"If we choose not to exercise early when the stock goes up once, the value of the option is {Cu:.3f}")
print(f"If we choose to exercise early when the stock goes up once, the payoff is {max(46 * u - K, 0):.3f}")
Cd = one_step_binomial(r = 0.06, dt = 0.5, u = u, d = d, Vu = max(46 * d * u - K, 0), Vd = max(46 * d * d -K, 0), delta = 0.02)
print(f"If we choose not to exercise early when the stock goes down once, the value of the option is {Cd:.3f}")
print(f"If we choose to exercise early when the stock goes up once, the payoff is {max(46 * d - K, 0):.3f}")
C = one_step_binomial(r = 0.06, dt = 0.5, u = u, d = d, Vu = max(Cu, max(46 * u - K, 0)), Vd = Cd, delta = 0.02)
print(f"If we choose not to exercise early, then the payoff is {C:.3f}")

If we choose not to exercise early when the stock goes up once, the value of the option is 38.197
If we choose to exercise early when the stock goes up once, the payoff is 38.204
If we choose not to exercise early when the stock goes down once, the value of the option is 19.678
If we choose to exercise early when the stock goes up once, the payoff is 19.500
If we choose not to exercise early, then the payoff is 27.199


**Quiz 3.6:**
Consider a one-period binomial model of 6 months. Assume the stock price is 63.00, $\sigma$ = 0.28, the annual continuously compounded risk free rate r = 5% and the stock's expected annual continuously compounded return is 14.0%. There are no dividends.

Assume the binomial tree is constructed using:

$$u = e^{(r-\delta)h+\sigma\sqrt{h}}$$
$$d = e^{(r-\delta)h-\sigma\sqrt{h}}$$

What is the true probability of the stock going up?

In [24]:
u = math.exp(0.05 * 0.5 + 0.28 * 0.5 ** 0.5)
d = math.exp(0.05 * 0.5 - 0.28 * 0.5 ** 0.5)
print(f"The probability that the stock will go up is {(math.exp(0.07) - d)/(u-d):.3f}")

The probability that the stock will go up is 0.566


**Quiz 3.7:**
For a two-period binomial model for stock prices, you are given:

- Each period is one year.
- The current price of a stock is 100
- The stock pays dividends continuously at an annual rate of 3%.
- u=1.148
- d=0.875
- The annual continuously compounded risk-free rate is 8%.

Calculate the current price of a two-year American call option on the stock with a strike price of 95.

**Quiz 3.8:**
True or False:  Option pricing does not depend on the expected return of the underlying stock.

**Quiz 3.9:**
The current price of a stock is 35. The stock pays dividends continuously at an annual rate of 2%. The stock’s volatility is 25%. Stock prices follow the binomial pricing model with the length of each period h being 3 months. The binomial tree is constructed using:

$$u = e^{(r-\delta)h+\sigma\sqrt{h}}$$
$$d = e^{(r-\delta)h-\sigma\sqrt{h}}$$

The continuously compounded risk-free interest rate is 4%. A 6-month American call option on the stock has a strike price of 30.

Using a two-period standard binomial model, calculate the price of the call option.

**Quiz 3.10:**
The stock price S = 100 and the annual continuously compounded risk-free rate is 5%. There are no dividends.

Consider a 1-year European call option strike at- the-money (strike price = current stock price)

If the volatility $\sigma$ is zero, what is the option worth?

In [26]:
(100 * math.exp(0.05) - 100) * math.exp(-0.05)

4.877057549928611